In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import numpy as np
import time
from sklearn.metrics.pairwise import cosine_similarity
import torch
import re
import sys

In [ ]:
df_cluster = pd.read_csv('All_Cluster_Tweet_Topic_2023.csv', dtype = 'str')
df_embedding = pd.read_csv('Cluster_Embedding_2023.csv')

In [ ]:
df_embedding

In [ ]:
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2", device='cuda:0')
model.max_seq_length = 280
model.do_lower_case =  True

In [ ]:
df_alt_fake_tweet_2024 = pd.read_csv('New_Fake_Tweets_2024.csv', dtype = 'str')

df_cluster2 = df_alt_fake_tweet_2024
df_cluster2 = df_cluster2.dropna(subset=['Tweet_ID'])
df_cluster2['Text_without_Link'] = df_cluster2['text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
df_cluster2['Text_without_Link']=df_cluster2['Text_without_Link'].str.replace('(@\w+.*?)',"")
df_cluster2['Text_without_Link'] = df_cluster2['Text_without_Link'].replace('', np.nan)
df_cluster2 = df_cluster2.dropna(subset=['Text_without_Link'])
df_cluster2 = df_cluster2[(df_cluster2.Text_Lang == 'en') | (df_cluster2.Text_Lang == 'hi')]
df_cluster2 = df_cluster2.dropna(subset=['Text_without_Link'])
df_cluster2 = df_cluster2.reset_index(drop = True)
df_cluster2['Text_Size'] = df_cluster2.Text_without_Link.apply(lambda x: len(x))
df_cluster2 = df_cluster2.loc[df_cluster2.Text_Size >= 20]
df_cluster2.Tweet_ID = df_cluster2.Tweet_ID.astype('int')

In [ ]:
i=0
all_text2 = df_cluster2.Text_without_Link.tolist()
embeddings2 = model.encode(all_text2[i*3000:(i+1)*3000])

In [ ]:
f_set = ['Tweet_ID']
for i in range(0,384):
    f_set.append('F' + str(i))
df_embedding2 = pd.DataFrame(columns=f_set)
df_embedding2['Tweet_ID'] =df_cluster2['Tweet_ID']

emb_df2 = pd.DataFrame(embeddings2)
df_embedding2.iloc[:,1:] = emb_df2.iloc[:,:]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
grouped_clusters = df_embedding.groupby('Cluster_ID')

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


grouped_clusters = df_embedding.groupby('Cluster_ID')

cluster_avg_cosine_sim = []

for cluster_id, group in grouped_clusters:
    embeddings = group.iloc[:, 2:-1].values  
    
    cosine_sim_matrix = cosine_similarity(embeddings)
    
    np.fill_diagonal(cosine_sim_matrix, np.nan)
    
    avg_cosine_similarity = np.nanmean(cosine_sim_matrix)
    
    cluster_avg_cosine_sim.append({'Cluster_ID': cluster_id, 'Avg_Cosine_Similarity': avg_cosine_similarity})

df_avg_cosine_similarity = pd.DataFrame(cluster_avg_cosine_sim)



In [ ]:
print(df_avg_cosine_similarity.Avg_Cosine_Similarity.mean())
print(df_avg_cosine_similarity.Avg_Cosine_Similarity.median())
print(df_avg_cosine_similarity.Avg_Cosine_Similarity.min())

In [ ]:
len(df_embedding.loc[:, '0':'383'])

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

df_embedding.Tweet_ID = df_embedding.Tweet_ID.astype('str')
cluster_embeddings = df_embedding.loc[:, '0':'383'].values
fake_embeddings = df_embedding2.loc[:, 'F0':'F383'].values

similarity_matrix = cosine_similarity(cluster_embeddings, fake_embeddings)

threshold = 0.95

matching_indices = np.where(similarity_matrix > threshold)

matches = []

for cluster_idx, fake_idx in zip(matching_indices[0], matching_indices[1]):
    cluster_row = df_embedding.iloc[cluster_idx]
    fake_row = df_embedding2.iloc[fake_idx]
    
    matches.append({
        'Cluster_ID': cluster_row['Cluster_ID'],
        'Tweet_ID_Cluster': cluster_row['Tweet_ID'],
        'Tweet_ID_Fake': fake_row['Tweet_ID'],
        'Similarity': similarity_matrix[cluster_idx, fake_idx]
    })

df_matches = pd.DataFrame(matches)



In [ ]:
df_matches.groupby('Cluster_ID').count()

In [ ]:
df_matches.loc[df_matches.Similarity >0.99]

In [ ]:
df_cluster.groupby('Cluster_ID').count()